# Calculating portfolio returns 
* This is the same exact file as portfolio investing but instead of using BTC, we have actual stock data

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import statsmodels
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
achc = pd.read_csv('ACHC.csv', parse_dates=['Date'])
achc = achc.sort_values(by='Date')
achc.set_index('Date', inplace=True)

In [ ]:
# calculate daily returns
achc['Returns'] = achc['Adj Close'].pct_change()
achc['Returns'].head()

In [ ]:
# visualize returns
plt.hist(achc['Returns'].dropna(), bins=75, density=False)
plt.show();

In order to build a backtest portfolio we need to have the returns of multiple assets in a single object

Our model portfolio consists of the following companies and the weight in our portfolio represented by percentages:
* Apple 12%
* Microsoft 15%
* Exxon Mobil 8%
* JP Morgan 9%
* Amazon 10%
* Facebook 5%
* AT&T 16%
* Tesla 15%
* Gold 10%

In [ ]:
# calculate mean returns
mean_return_daily = np.mean(achc['Returns'])
print(mean_return_daily)

In [ ]:
# calculate implied annualized average return assuming 252 trading days in a year
mean_return_annualized = ((1 + np.mean(achc['Returns'])) ** 252) - 1
print(mean_return_annualized)

In [ ]:
# standard deviation
sigma_daily = np.std(achc['Returns'])
print(sigma_daily)

In [ ]:
# calculate the variance, simply square the standard deviation
variance_daily = np.std(achc['Returns']) ** 2
print(variance_daily)

In [ ]:
# calculate annualized standard deviation
sigma_annualized = sigma_daily * np.sqrt(252)
print(sigma_annualized)

# calculate annualized variance
variance_annualized = sigma_annualized ** 2
print(variance_annualized)

## Skewness and Kurtosis

In [ ]:
from scipy.stats import skew

clean_returns = (achc['Returns'].dropna())

# calculate the third moment(skewness) of the returns distribution
returns_skewness = skew(clean_returns)
print(returns_skewness)

* YIKES
A negative skew is a right leaning curve, and a left leaning curve is a positive skew.
This portfolio is a wreck

In [ ]:
# import kurtosis
from scipy.stats import kurtosis

# calculate the excess kurtosis of the returns distribution
excess_kurtosis = kurtosis(clean_returns)
print(excess_kurtosis)

# derive the true fourth moment of the returns distribution
fourth_moment = excess_kurtosis + 3
print(fourth_moment)

## Statistical tests for normality

### Shapiro-Wlk test

In [ ]:
# import shapiro from scipy.stats
from scipy.stats import shapiro

# run shapiro wilk test
shapiro_results = shapiro(clean_returns)
print('Shapiro results:', shapiro_results)

# extract p_value from the shapiro results
p_value = shapiro_results[1]
print('P-value: ', p_value)

In [ ]:
# define portoflio weights as numpy array
RandomPortfolios = np.array([0.12, 0.15, 0.08, 0.09, 0.10, 0.05, 0.16, 0.15, 0.10])

# calculate the weighted stock returns
#WeightedReturns = achc['Returns'].mul(RandomPortfolios)

# calculate the portfolio returns
#achc['Portfolio'] = WeightedReturns.sum()

# plot cumulative portfolio returns over time
#CumulativeReturns = ((1 + achc['Portfolio']).cumprod()-1)
#CumulativeReturns.plot()
#plt.show();

## Sharpe ratios

The sharpe ratio is a metric of risk adjusted return.
The ratio is useful to determine how much risk is being taken to achieve a certain level of return.

In [ ]:
# risk free rate
#risk_free = 0

# calculate sharpe ratio for each asset
#RandomPortfolios['Sharpe'] = (RandomPortfolios['Returns'] - risk_free) / (RandomPortfolios['Volatility']);

In [ ]:
import statsmodels.formula.api as smf 

# Define the regression formula
CAPM_model = smf.ols(formula='Portfolio_Excess ~ Market_Excess', data=achc)

# Fit the regression
CAPM_fit = CAPM_model.fit()
print(CAPM_fit.rsquared_adj)

# Extract the beta
regression_beta = CAPM_fit.params['Market_Excess']
print(regression_beta);

In [ ]:
%%capture
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
achc_model = smf.ols(formula='excess_kurtosis ~ Market_Excess + SMB + HML', data=achc)

# Fit the regression
FamaFrench_fit = FamaFrench_model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = FamaFrench_fit.rsquared_adj
print(regression_adj_rsq)

In [ ]:
# Extract the p-value of the SMB factor
smb_pval = FamaFrench_fit.pvalues['SMB']

# If the p-value is significant, print significant
if smb_pval < 0.05:
    significant_msg = 'significant'
else:
    significant_msg = 'not significant'

# Print the SMB coefficient
smb_coeff = FamaFrench_fit.params['SMB']
print("The SMB coefficient is ", smb_coeff, " and is ", significant_msg)